In [94]:
import csv
import pandas as pd
from pandas import *
from __future__ import division
from pandas.tseries.offsets import *
import dateutil.rrule as dr
import dateutil.parser as dp
import dateutil.relativedelta as drel
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [95]:
pd.options.mode.chained_assignment = None

## Loading files

In [96]:
#spd = read_csv('http://real-chart.finance.yahoo.com/table.csv?s=%5EGSPC&a=00&b=1&c=1974&d=05&e=9&f=2015&g=d&ignore=.csv',
#               delimiter=',', header=0, 
#               names=['date','o','h','l','close','vol','c'])
#spw = read_csv('http://real-chart.finance.yahoo.com/table.csv?s=%5EGSPC&a=00&b=1&c=1974&d=05&e=9&f=2015&g=w&ignore=.csv',
#               delimiter=',', header=0, 
#               names=['date','o','h','l','close','vol','c'])

#http://real-chart.finance.yahoo.com/table.csv?s=DAL&a=04&b=3&c=2008&d=05&e=9&f=2015&g=d&ignore=.csv
#http://real-chart.finance.yahoo.com/table.csv?s=%5EGSPC&a=00&b=1&c=1974&d=05&e=9&f=2015&g=d&ignore=.csv

In [97]:
spd = read_csv('spd.csv', delimiter=',', header=0)
spw = read_csv('spw.csv', delimiter=',', header=0)

In [98]:
spd.rename(columns={'Unnamed: 0':'nrow'}, inplace=True)
spw.rename(columns={'Unnamed: 0':'nrow'}, inplace=True)

In [99]:
spd['date'] = pd.to_datetime(spd['date'])
spw['date'] = pd.to_datetime(spw['date'])
print type(spd.date[0]), type(spw.date[0])

<class 'pandas.tslib.Timestamp'> <class 'pandas.tslib.Timestamp'>


In [100]:
spd.head(3)

,nrow,date,o,h,l,close,vol,c
0,0,2015-06-19,2121.060059,2121.639893,2109.449951,2109.989990,4449810000,2109.989990
1,1,2015-06-18,2101.580078,2126.649902,2101.580078,2121.239990,3520360000,2121.239990
2,2,2015-06-17,2097.399902,2106.790039,2088.860107,2100.439941,3222240000,2100.439941


In [101]:
spd.tail(3)

,nrow,date,o,h,l,close,vol,c
10459,10459,1974-01-04,99.800003,100.699997,97.699997,98.900002,21700000,98.900002
10460,10460,1974-01-03,98.019997,100.940002,98.019997,99.800003,24850000,99.800003
10461,10461,1974-01-02,97.550003,98.379997,96.250000,97.680000,12060000,97.680000


In [102]:
spw.head(3)

,nrow,date,o,h,l,close,vol,c
0,0,2015-06-15,2091.340088,2126.649902,2072.489990,2109.989990,3434776000,2109.989990
1,1,2015-06-08,2092.340088,2115.020020,2072.139893,2094.110107,3042810000,2094.110107
2,2,2015-06-01,2108.639893,2121.919922,2085.669922,2092.830078,3120956000,2092.830078


In [103]:
spw.tail(3)

,nrow,date,o,h,l,close,vol,c
2161,2161,1974-01-14,93.660004,98.349998,92.349998,95.559998,16060000,95.559998
2162,2162,1974-01-07,98.900002,99.309998,91.620003,93.660004,17296000,93.660004
2163,2163,1974-01-02,97.550003,100.940002,96.250000,98.900002,19536600,98.900002


In [104]:
len(spd)

10462

In [105]:
len(spw)

2164

In [106]:
spd.date[spd.date == dp.parse("20/03/2008")] = pd.to_datetime(dp.parse("21/03/2008"))

In [107]:
spd.date[spd.date == dp.parse("21/04/2011")] = pd.to_datetime(dp.parse("22/04/2011"))

In [108]:
spd.date[spd.date == dp.parse("20/04/2000")] = pd.to_datetime(dp.parse("21/04/2000"))

In [109]:
spd.date[spd.date == dp.parse("19/04/1984")] = pd.to_datetime(dp.parse("20/04/1984"))

In [110]:
for i in range(len(spw.date)):
    spw.date[i] = spw.date[i] + DateOffset(weekday = 4) + DateOffset(days = 1) - BDay()
#    if spw.date[i] == dp.parse("21/03/2008"):
#        spw.date[i] = pd.to_datetime(dp.parse("20/03/2008"))

In [111]:
spw.head(3)

,nrow,date,o,h,l,close,vol,c
0,0,2015-06-19,2091.340088,2126.649902,2072.489990,2109.989990,3434776000,2109.989990
1,1,2015-06-12,2092.340088,2115.020020,2072.139893,2094.110107,3042810000,2094.110107
2,2,2015-06-05,2108.639893,2121.919922,2085.669922,2092.830078,3120956000,2092.830078


In [112]:
spw.tail(3)

,nrow,date,o,h,l,close,vol,c
2161,2161,1974-01-18,93.660004,98.349998,92.349998,95.559998,16060000,95.559998
2162,2162,1974-01-11,98.900002,99.309998,91.620003,93.660004,17296000,93.660004
2163,2163,1974-01-04,97.550003,100.940002,96.250000,98.900002,19536600,98.900002


## Moving Averages

In [113]:
d12 = []
d24 = []
d48 = []
d72 = []
d96 = []
d192 = []
d288 = []
for i in range(len(spd)-288):
    d12.append(sum(spd.c[i:(i+12)])/12)
    d24.append(sum(spd.c[i:(i+24)])/24)
    d48.append(sum(spd.c[i:(i+48)])/48)
    d72.append(sum(spd.c[i:(i+72)])/72)
    d96.append(sum(spd.c[i:(i+96)])/96)
    d192.append(sum(spd.c[i:(i+192)])/192)
    d288.append(sum(spd.c[i:(i+288)])/288)
    
d192[-1]

75.564843718750026

In [114]:
w12 = []
w24 = []
w48 = []
w72 = []
w96 = []
w192 = []
w288 = []
for i in range(len(spw)-288):
    w12.append(sum(spw.c[i:(i+12)])/12)
    w24.append(sum(spw.c[i:(i+24)])/24)
    w48.append(sum(spw.c[i:(i+48)])/48)
    w72.append(sum(spw.c[i:(i+72)])/72)
    w96.append(sum(spw.c[i:(i+96)])/96)
    w192.append(sum(spw.c[i:(i+192)])/192)
    w288.append(sum(spw.c[i:(i+288)])/288)
    
w192[-1]

98.690937416666671

## Indicators

In [115]:
ltd = []
for i in range(len(spd)-288):
    t12 = ((spd.c[i]/(sum(spd.c[(i+6):(i+12)])/6)-1)*100)
    t24 = ((spd.c[i]/(sum(spd.c[(i+18):(i+24)])/6)-1)*100)
    t48 = ((spd.c[i]/(sum(spd.c[(i+42):(i+48)])/6)-1)*100)
    t72 = ((spd.c[i]/(sum(spd.c[(i+66):(i+72)])/6)-1)*100)
    t96 = ((spd.c[i]/(sum(spd.c[(i+90):(i+96)])/6)-1)*100)
    t192 = ((spd.c[i]/(sum(spd.c[(i+186):(i+192)])/6)-1)*100)
    t288 = ((spd.c[i]/(sum(spd.c[(i+282):(i+288)])/6)-1)*100)
    ltd.append((t12+t24+t48+t72+t96+t192+t288)/7)
ltd.extend([0]*288)

In [116]:
ltw = []
for i in range(len(spw)-288):
    t12 = ((spw.c[i]/(sum(spw.c[(i+6):(i+12)])/6)-1)*100)
    t24 = ((spw.c[i]/(sum(spw.c[(i+18):(i+24)])/6)-1)*100)
    t48 = ((spw.c[i]/(sum(spw.c[(i+42):(i+48)])/6)-1)*100)
    t72 = ((spw.c[i]/(sum(spw.c[(i+66):(i+72)])/6)-1)*100)
    t96 = ((spw.c[i]/(sum(spw.c[(i+90):(i+96)])/6)-1)*100)
    t192 = ((spw.c[i]/(sum(spw.c[(i+186):(i+192)])/6)-1)*100)
    t288 = ((spw.c[i]/(sum(spw.c[(i+282):(i+288)])/6)-1)*100)
    ltw.append((t12+t24+t48+t72+t96+t192+t288)/7)
ltw.extend([0]*288)

In [117]:
print max(ltd), min(ltd), max(ltw), min (ltw)

23.3328697298 -35.6366112059 71.7672905826 -41.5810392106


In [118]:
print ltd.index(max(ltd)), ltd.index(min(ltd)), ltw.index(max(ltw)), ltw.index(min(ltw))

8099 1654 883 328


In [119]:
lc288d = []
lc72d = []
for i in range(len(spd)-288):
    min12 = ((spd.c[i]/min(spd.c[(i+6):(i+12)]))-1)*100
    max12 = ((spd.c[i]/max(spd.c[(i+6):(i+12)]))-1)*100
    min24 = ((spd.c[i]/min(spd.c[(i+18):(i+24)]))-1)*100
    max24 = ((spd.c[i]/max(spd.c[(i+18):(i+24)]))-1)*100
    min48 = ((spd.c[i]/min(spd.c[(i+42):(i+48)]))-1)*100
    max48 = ((spd.c[i]/max(spd.c[(i+42):(i+48)]))-1)*100
    min72 = ((spd.c[i]/min(spd.c[(i+66):(i+72)]))-1)*100
    max72 = ((spd.c[i]/max(spd.c[(i+66):(i+72)]))-1)*100
    min96 = ((spd.c[i]/min(spd.c[(i+90):(i+96)]))-1)*100
    max96 = ((spd.c[i]/max(spd.c[(i+90):(i+96)]))-1)*100
    min192 = ((spd.c[i]/min(spd.c[(i+186):(i+192)]))-1)*100
    max192 = ((spd.c[i]/max(spd.c[(i+186):(i+192)]))-1)*100
    min288 = ((spd.c[i]/min(spd.c[(i+282):(i+288)]))-1)*100
    max288 = ((spd.c[i]/max(spd.c[(i+282):(i+288)]))-1)*100
    
    clong288 = 0
    clong72 = 0
    cshort288 = 0
    cshort72 = 0
    
    contatl = 0
    if spd.c[i] < sum(spd.c[i:i+12])/12:
        contatl += 1
    if spd.c[i] > spd.c[i+12]:
        contatl += 1
    if max12 > 0:
        contatl += 1
    contats = 0
    if spd.c[i] > sum(spd.c[i:i+12])/12:
        contats -= 1
    if spd.c[i] < spd.c[i+12]:
        contats -= 1
    if min12 < 0:
        contats -= 1
    if contatl == 3 or contats == -3:
        c12 = (((sum(spd.c[i:(i+12)])/12)/spd.c[i])-1)*100
    else:
        c12 = 0
    if contatl == 3:
        clong288 += 1
        clong72 += 1
    if contats == -3:
        cshort288 += 1
        cshort72 += 1
        
    contatl = 0
    if spd.c[i] < sum(spd.c[i:i+24])/24:
        contatl +=1
    if spd.c[i] > spd.c[i+24]:
        contatl +=1
    if max24 > 0:
        contatl +=1
    contats = 0
    if spd.c[i] > sum(spd.c[i:i+24])/24:
        contats -=1
    if spd.c[i] < spd.c[i+24]:
        contats -=1
    if min24 < 0:
        contats -=1
    if contatl == 3 or contats == -3:
        c24 = (((sum(spd.c[i:(i+24)])/24)/spd.c[i])-1)*100
    else:
        c24 = 0
    if contatl == 3:
        clong288 +=1
        clong72 +=1
    if contats == -3:
        cshort288 +=1
        cshort72 +=1
        
    contatl = 0
    if spd.c[i] < sum(spd.c[i:i+48])/48:
        contatl +=1
    if spd.c[i] > spd.c[i+48]:
        contatl +=1
    if max48 > 0:
        contatl +=1
    contats = 0
    if spd.c[i] > sum(spd.c[i:i+48])/48:
        contats -=1
    if spd.c[i] < spd.c[i+48]:
        contats -=1
    if min48 < 0:
        contats -=1
    if contatl == 3 or contats == -3:
        c48 = (((sum(spd.c[i:(i+48)])/48)/spd.c[i])-1)*100
    else:
        c48 = 0    
    if contatl == 3:
        clong288 +=1
        clong72 +=1
    if contats == -3:
        cshort288 +=1
        cshort72 +=1
    
    contatl = 0
    if spd.c[i] < sum(spd.c[i:i+72])/72:
        contatl +=1
    if spd.c[i] > spd.c[i+72]:
        contatl +=1
    if max72 > 0:
        contatl +=1
    contats = 0
    if spd.c[i] > sum(spd.c[i:i+72])/72:
        contats -=1
    if spd.c[i] < spd.c[i+72]:
        contats -=1
    if min72 < 0:
        contats -=1
    if contatl == 3 or contats == -3:
        c72 = (((sum(spd.c[i:(i+72)])/72)/spd.c[i])-1)*100
    else:
        c72 = 0  
    if contatl == 3:
        clong288 +=1
        clong72 +=1
    if contats == -3:
        cshort288 +=1
        cshort72 +=1
    
    contatl = 0
    if spd.c[i] < sum(spd.c[i:i+96])/96:
        contatl +=1
    if spd.c[i] > spd.c[i+96]:
        contatl +=1
    if max96 > 0:
        contatl +=1
    contats = 0
    if spd.c[i] > sum(spd.c[i:i+96])/96:
        contats -=1
    if spd.c[i] < spd.c[i+96]:
        contats -=1
    if min96 < 0:
        contats -=1
    if contatl == 3 or contats == -3:
        c96 = (((sum(spd.c[i:(i+96)])/96)/spd.c[i])-1)*100
    else:
        c96 = 0
    if contatl == 3:
        clong288 +=1
    if contats == -3:
        cshort288 +=1
        
    contatl = 0
    if spd.c[i] < sum(spd.c[i:i+192])/192:
        contatl +=1
    if spd.c[i] > spd.c[i+192]:
        contatl +=1
    if max192 > 0:
        contatl +=1
    contats = 0
    if spd.c[i] > sum(spd.c[i:i+192])/192:
        contats -=1
    if spd.c[i] < spd.c[i+192]:
        contats -=1
    if min192 < 0:
        contats -=1
    if contatl == 3 or contats == -3:
        c192 = (((sum(spd.c[i:(i+192)])/192)/spd.c[i])-1)*100
    else:
        c192 = 0      
    if contatl == 3:
        clong288 +=1
    if contats == -3:
        cshort288 +=1
        
    contatl = 0
    if spd.c[i] < sum(spd.c[i:i+288])/288:
        contatl +=1
    if spd.c[i] > spd.c[i+288]:
        contatl +=1
    if max288 > 0:
        contatl +=1
    contats = 0
    if spd.c[i] > sum(spd.c[i:i+288])/288:
        contats -=1
    if spd.c[i] < spd.c[i+288]:
        contats -=1
    if min288 < 0:
        contats -=1
    if contatl == 3 or contats == -3:
        c288 = (((sum(spd.c[i:(i+288)])/288)/spd.c[i])-1)*100
    else:
        c288 = 0
    if contatl == 3:
        clong288 +=1
    if contats == -3:
        cshort288 += 1

    if (clong288 + cshort288) > 0:
        ctot288 = (clong288 + cshort288)
    else:
        ctot288 = 1
    lc288d.append((c12+c24+c48+c72+c96+c192+c288)/ctot288)
    
    if (clong72 + cshort72) > 0:
        ctot72 = (clong72 + cshort72)
    else:
        ctot72 = 1
    lc72d.append((c12+c24+c48+c72)/ctot72)
lc288d.extend([0]*288)
lc72d.extend([0]*288)

In [120]:
print max(lc288d), min(lc288d), max(lc72d), min(lc72d)

17.8730823251 -15.6973020039 4.46218418268 -6.65612308983


In [121]:
print lc288d.index(max(lc288d)), lc288d.index(min(lc288d)), lc72d.index(max(lc72d)), lc72d.index(min(lc72d))

6958 1409 9246 1554


In [122]:
print spd.date[1644]

2008-12-05 00:00:00


In [123]:
print lc288d[0], lc72d[0]

0.0 0.0


In [124]:
lc288w = []
lc72w = []
for i in range(len(spw)-288):
    min12 = ((spw.c[i]/min(spw.c[(i+6):(i+12)]))-1)*100
    max12 = ((spw.c[i]/max(spw.c[(i+6):(i+12)]))-1)*100
    min24 = ((spw.c[i]/min(spw.c[(i+18):(i+24)]))-1)*100
    max24 = ((spw.c[i]/max(spw.c[(i+18):(i+24)]))-1)*100
    min48 = ((spw.c[i]/min(spw.c[(i+42):(i+48)]))-1)*100
    max48 = ((spw.c[i]/max(spw.c[(i+42):(i+48)]))-1)*100
    min72 = ((spw.c[i]/min(spw.c[(i+66):(i+72)]))-1)*100
    max72 = ((spw.c[i]/max(spw.c[(i+66):(i+72)]))-1)*100
    min96 = ((spw.c[i]/min(spw.c[(i+90):(i+96)]))-1)*100
    max96 = ((spw.c[i]/max(spw.c[(i+90):(i+96)]))-1)*100
    min192 = ((spw.c[i]/min(spw.c[(i+186):(i+192)]))-1)*100
    max192 = ((spw.c[i]/max(spw.c[(i+186):(i+192)]))-1)*100
    min288 = ((spw.c[i]/min(spw.c[(i+282):(i+288)]))-1)*100
    max288 = ((spw.c[i]/max(spw.c[(i+282):(i+288)]))-1)*100
    
    clong288 = 0
    clong72 = 0
    cshort288 = 0
    cshort72 = 0
    
    contatl = 0
    if spw.c[i] < sum(spw.c[i:i+12])/12:
        contatl += 1
    if spw.c[i] > spw.c[i+12]:
        contatl += 1
    if max12 > 0:
        contatl += 1
    contats = 0
    if spw.c[i] > sum(spw.c[i:i+12])/12:
        contats -= 1
    if spw.c[i] < spw.c[i+12]:
        contats -= 1
    if min12 < 0:
        contats -= 1
    if contatl == 3 or contats == -3:
        c12 = (((sum(spw.c[i:(i+12)])/12)/spw.c[i])-1)*100
    else:
        c12 = 0
    if contatl == 3:
        clong288 += 1
        clong72 += 1
    if contats == -3:
        cshort288 += 1
        cshort72 += 1
        
    contatl = 0
    if spw.c[i] < sum(spw.c[i:i+24])/24:
        contatl +=1
    if spw.c[i] > spw.c[i+24]:
        contatl +=1
    if max24 > 0:
        contatl +=1
    contats = 0
    if spw.c[i] > sum(spw.c[i:i+24])/24:
        contats -=1
    if spw.c[i] < spw.c[i+24]:
        contats -=1
    if min24 < 0:
        contats -=1
    if contatl == 3 or contats == -3:
        c24 = (((sum(spw.c[i:(i+24)])/24)/spw.c[i])-1)*100
    else:
        c24 = 0
    if contatl == 3:
        clong288 +=1
        clong72 +=1
    if contats == -3:
        cshort288 +=1
        cshort72 +=1
        
    contatl = 0
    if spw.c[i] < sum(spw.c[i:i+48])/48:
        contatl +=1
    if spw.c[i] > spw.c[i+48]:
        contatl +=1
    if max48 > 0:
        contatl +=1
    contats = 0
    if spw.c[i] > sum(spw.c[i:i+48])/48:
        contats -=1
    if spw.c[i] < spw.c[i+48]:
        contats -=1
    if min48 < 0:
        contats -=1
    if contatl == 3 or contats == -3:
        c48 = (((sum(spw.c[i:(i+48)])/48)/spw.c[i])-1)*100
    else:
        c48 = 0    
    if contatl == 3:
        clong288 +=1
        clong72 +=1
    if contats == -3:
        cshort288 +=1
        cshort72 +=1
    
    contatl = 0
    if spw.c[i] < sum(spw.c[i:i+72])/72:
        contatl +=1
    if spw.c[i] > spw.c[i+72]:
        contatl +=1
    if max72 > 0:
        contatl +=1
    contats = 0
    if spw.c[i] > sum(spw.c[i:i+72])/72:
        contats -=1
    if spw.c[i] < spw.c[i+72]:
        contats -=1
    if min72 < 0:
        contats -=1
    if contatl == 3 or contats == -3:
        c72 = (((sum(spw.c[i:(i+72)])/72)/spw.c[i])-1)*100
    else:
        c72 = 0  
    if contatl == 3:
        clong288 +=1
        clong72 +=1
    if contats == -3:
        cshort288 +=1
        cshort72 +=1
    
    contatl = 0
    if spw.c[i] < sum(spw.c[i:i+96])/96:
        contatl +=1
    if spw.c[i] > spw.c[i+96]:
        contatl +=1
    if max96 > 0:
        contatl +=1
    contats = 0
    if spw.c[i] > sum(spw.c[i:i+96])/96:
        contats -=1
    if spw.c[i] < spw.c[i+96]:
        contats -=1
    if min96 < 0:
        contats -=1
    if contatl == 3 or contats == -3:
        c96 = (((sum(spw.c[i:(i+96)])/96)/spw.c[i])-1)*100
    else:
        c96 = 0
    if contatl == 3:
        clong288 +=1
    if contats == -3:
        cshort288 +=1
        
    contatl = 0
    if spw.c[i] < sum(spw.c[i:i+192])/192:
        contatl +=1
    if spw.c[i] > spw.c[i+192]:
        contatl +=1
    if max192 > 0:
        contatl +=1
    contats = 0
    if spw.c[i] > sum(spw.c[i:i+192])/192:
        contats -=1
    if spw.c[i] < spw.c[i+192]:
        contats -=1
    if min192 < 0:
        contats -=1
    if contatl == 3 or contats == -3:
        c192 = (((sum(spw.c[i:(i+192)])/192)/spw.c[i])-1)*100
    else:
        c192 = 0      
    if contatl == 3:
        clong288 +=1
    if contats == -3:
        cshort288 +=1
        
    contatl = 0
    if spw.c[i] < sum(spw.c[i:i+288])/288:
        contatl +=1
    if spw.c[i] > spw.c[i+288]:
        contatl +=1
    if max288 > 0:
        contatl +=1
    contats = 0
    if spw.c[i] > sum(spw.c[i:i+288])/288:
        contats -=1
    if spw.c[i] < spw.c[i+288]:
        contats -=1
    if min288 < 0:
        contats -=1
    if contatl == 3 or contats == -3:
        c288 = (((sum(spw.c[i:(i+288)])/288)/spw.c[i])-1)*100
    else:
        c288 = 0
    if contatl == 3:
        clong288 +=1
    if contats == -3:
        cshort288 += 1

    if (clong288 + cshort288) > 0:
        ctot288 = (clong288 + cshort288)
    else:
        ctot288 = 1
    lc288w.append((c12+c24+c48+c72+c96+c192+c288)/ctot288)
    
    if (clong72 + cshort72) > 0:
        ctot72 = (clong72 + cshort72)
    else:
        ctot72 = 1
    lc72w.append((c12+c24+c48+c72)/ctot72)
lc288w.extend([0]*288)
lc72w.extend([0]*288)

In [125]:
print max(lc288w), min(lc288w), max(lc72w), min(lc72w)

42.2414306189 -15.0945826014 9.81403381868 -13.8656712573


In [126]:
print lc288w.index(max(lc288w)), lc288w.index(min(lc288w)), lc72w.index(max(lc72w)), lc72w.index(min(lc72w))

664 216 1428 286


In [127]:
print spw.date[214]

2011-05-13 00:00:00


In [128]:
print lc288w[0], lc72w[0]

0.0 0.0


In [129]:
ldd = []
for i in range(len(spd)-288):
    ld = (((d12[i]*12)+(d24[i]*24)+(d48[i]*48)+(d72[i]*72)+(d96[i]*96)+(d192[i]*192)+(d288[i]*288))/732)
    ldd.append(((ld/spd.c[i])-1)*100)
ldd.extend([0]*288)

In [130]:
print max(ldd), min(ldd)

60.3588415804 -16.6628292676


In [131]:
print ldd.index(max(ldd)), ldd.index(min(ldd))

1654 8227


In [132]:
print spd.date[8217]

1982-11-17 00:00:00


In [133]:
print ldd[0]

-2.61936157337


In [134]:
ldw = []
for i in range(len(spw)-288):
    ld = (((w12[i]*12)+(w24[i]*24)+(w48[i]*48)+(w72[i]*72)+(w96[i]*96)+(w192[i]*192)+(w288[i]*288))/732)
    ldw.append(((ld/spw.c[i])-1)*100)
ldw.extend([0]*288)

In [135]:
print max(ldw), min(ldw)

79.734825175 -34.8467525679


In [136]:
print ldw.index(max(ldw)), ldw.index(min(ldw))

328 898


In [137]:
print spw.date[326]

2009-03-20 00:00:00


In [138]:
print ldw[0]

-18.1499495896


In [139]:
lmd = []
for i in range(len(spd)-288):
    lm = 0
    if spd.c[i] > d96[i] and spd.c[i] > d288[i]:
        lm = 1
    elif spd.c[i] < d96[i] and spd.c[i] < d288[i]:
        lm = -1
    lmd.append(lm)
lmd.extend([0]*288)

In [140]:
print max(lmd), min(lmd)

1 -1


In [141]:
print sum(lmd)/len(lmd)

0.421047600841


In [142]:
print lmd[0]

1


In [143]:
lmw = []
for i in range(len(spw)-288):
    lm = 0
    if spw.c[i] > w96[i] and spw.c[i] > w288[i]:
        lm = 1
    elif spw.c[i] < w96[i] and spw.c[i] < w288[i]:
        lm = -1
    lmw.append(lm)    
lmw.extend([0]*288)

In [144]:
print max(lmw), min(lmw)

1 -1


In [145]:
print sum(lmw)/len(lmw)

0.551756007394


In [146]:
print lmw[0]

1


In [147]:
spd['lixt'] = ltd
spd['lixc72'] = lc72d
spd['lixc288'] = lc288d
spd['lixd'] = ldd
spd['lixm'] = lmd
spd.head(3)

,nrow,date,o,h,l,close,vol,c,lixt,lixc72,lixc288,lixd,lixm
0,0,2015-06-19,2121.060059,2121.639893,2109.449951,2109.989990,4449810000,2109.989990,3.371532,0,0,-2.619362,1
1,1,2015-06-18,2101.580078,2126.649902,2101.580078,2121.239990,3520360000,2121.239990,3.950901,0,0,-3.166602,1
2,2,2015-06-17,2097.399902,2106.790039,2088.860107,2100.439941,3222240000,2100.439941,3.012487,0,0,-2.246367,1


In [148]:
spw['lixt'] = ltw
spw['lixc72'] = lc72w
spw['lixc288'] = lc288w
spw['lixd'] = ldw
spw['lixm'] = lmw
spw.head(3)

,nrow,date,o,h,l,close,vol,c,lixt,lixc72,lixc288,lixd,lixm
0,0,2015-06-19,2091.340088,2126.649902,2072.489990,2109.989990,3434776000,2109.989990,30.155243,0,0,-18.149950,1
1,1,2015-06-12,2092.340088,2115.020020,2072.139893,2094.110107,3042810000,2094.110107,29.196579,0,0,-17.718657,1
2,2,2015-06-05,2108.639893,2121.919922,2085.669922,2092.830078,3120956000,2092.830078,29.583573,0,0,-17.847919,1


## Building samples dataset (predict all 3 months of the quarter)

In [149]:
exp = []
d = dp.parse("21/03/2015")
while d > dp.parse("20/03/1980"):
    d = d - WeekOfMonth(weekday = 4, week = 2) + DateOffset(days = 1) - BDay()
#    if d != dp.parse("21/09/2001"):
    exp.append(d)
    d = d - DateOffset(days = 80)

len(exp)

141

In [150]:
q1c = []
q1cstd = []
q1high = []
q1low = []
q1range = []
q1middle = []
q1pos = []
q0c = []
q0cstd = []
q0high = []
q0low = []
q0spanmean = []
q0spanstd = []
q0lixt = []
q0lixc72 = []
q0lixc288 = []
q0lixd = []
q0lixm = []
q0lixtv = []
q0lixc72v = []
q0lixc288v = []
q0lixdv = []
q0lixmv = []
m0c = []
m0cstd = []
m0high = []
m0low = []
m0spanmean = []
m0spanstd = []
m0lixt = []
m0lixc72 = []
m0lixc288 = []
m0lixd = []
m0lixm = []
m0lixtv = []
m0lixc72v = []
m0lixc288v = []
m0lixdv = []
m0lixmv = []

In [151]:
for q1end in exp:
    q1start = q1end - DateOffset(weeks = 13)
    m0start = q1start - DateOffset(weeks = 4)
    q0start = q1start - DateOffset(weeks = 13)
    q1indend = spw.nrow[spw.date == q1end]
    
    q1cstart = float(spw.c[q1indend + 13])
    q1cend = float(spw.c[q1indend])    
    q1cv = ((q1cend/q1cstart)-1)*100
    q1c.append(q1cv)   
    
    q1clist = spd.c[(spd.date >= q1start) & (spd.date<= q1end)]
    q1clist = q1clist.reset_index(drop=True)
    q1cvars = []
    for i in range(len(q1clist)):
        if i < len(q1clist)-1:
            q1cvars.append(((q1clist[i]/q1clist[i+1])-1)*100)
    q1cstd.append(np.std(q1cvars))
    
    q1chigh = ((np.max(q1clist)/q1cstart)-1)*100
    q1high.append(q1chigh)
    
    q1clow = ((np.min(q1clist)/q1cstart)-1)*100
    q1low.append(q1clow)
    
    q1crange = q1chigh - q1clow
    q1range.append(q1crange)
    
    q1cmiddle = q1chigh - (q1crange / 2)
    q1middle.append(q1cmiddle)
    
    q1cpos = (((q1cv - q1clow)/q1crange)-0.5)*200
    q1pos.append(q1cpos)
    
    q1indstart = spw.nrow[spw.date == q1start]
    
    q0cstart = float(spw.c[q1indstart + 13])
    q0cend = float(spw.c[q1indstart])    
    q0cv = ((q0cend/q0cstart)-1)*100
    q0c.append(q0cv)
    
    q0clist = spd.c[(spd.date >= q0start) & (spd.date<= q1start)]
    q0clist = q0clist.reset_index(drop=True)
    q0cvars = []
    for i in range(len(q0clist)):
        if i < len(q0clist)-1:
            q0cvars.append(((q0clist[i]/q0clist[i+1])-1)*100)
    q0cstd.append(np.std(q0cvars))
    
    q0hlist = spd.h[(spd.date >= q0start) & (spd.date<= q1start)]
    q0hlist = q0hlist.reset_index(drop=True)
    q0hvars = []
    for i in range(len(q0hlist)):
        q0hvars.append(((q0hlist[i]/q0clist[i])-1)*100)
    q0high.append(np.mean(q0hvars))
    
    q0llist = spd.l[(spd.date >= q0start) & (spd.date<= q1start)]
    q0llist = q0llist.reset_index(drop=True)
    q0lvars = []
    for i in range(len(q0llist)):
        q0lvars.append(((q0llist[i]/q0clist[i])-1)*100)
    q0low.append(np.mean(q0lvars))
    
    q0svars = []
    for i in range(len(q0llist)):
        q0svars.append(((q0hlist[i]/q0llist[i])-1)*100)
    q0spanmean.append(np.mean(q0svars))
    q0spanstd.append(np.std(q0svars))

    lixt1 = float((spw.lixt[spw.date == q1start]))
    lixt0 = float((spw.lixt[spw.date == q0start]))
    lixtm1 = float((spd.lixt[spd.date == q1start]))
    lixtm0 = float((spd.lixt[spd.date == m0start]))
    q0lixt.append(float(lixt1))
    q0lixtv.append(float(lixt1 - lixt0))
    m0lixt.append(float(lixtm1))
    m0lixtv.append(float(lixtm1 - lixtm0))
    
    lixc721 = float((spw.lixc72[spw.date == q1start]))
    lixc720 = float((spw.lixc72[spw.date == q0start]))
    lixc72m1 = float((spd.lixc72[spd.date == q1start]))
    lixc72m0 = float((spd.lixc72[spd.date == m0start]))
    q0lixc72.append(float(lixc721))
    q0lixc72v.append(float(lixc721 - lixc720))
    m0lixc72.append(float(lixc72m1))
    m0lixc72v.append(float(lixc72m1 - lixc72m0))
    
    lixc2881 = float((spw.lixc288[spw.date == q1start]))
    lixc2880 = float((spw.lixc288[spw.date == q0start]))
    lixc288m1 = float((spd.lixc288[spd.date == q1start]))
    lixc288m0 = float((spd.lixc288[spd.date == m0start]))
    q0lixc288.append(float(lixc2881))
    q0lixc288v.append(float(lixc2881 - lixc2880))
    m0lixc288.append(float(lixc288m1))
    m0lixc288v.append(float(lixc288m1 - lixc288m0))
    
    lixd1 = float((spw.lixd[spw.date == q1start]))
    lixd0 = float((spw.lixd[spw.date == q0start]))
    lixdm1 = float((spd.lixd[spd.date == q1start]))
    lixdm0 = float((spd.lixd[spd.date == m0start]))
    q0lixd.append(float(lixd1))
    q0lixdv.append(float(lixd1 - lixd0))
    m0lixd.append(float(lixdm1))
    m0lixdv.append(float(lixdm1 - lixdm0))
    
    lixm1 = float((spw.lixm[spw.date == q1start]))
    lixm0 = float((spw.lixm[spw.date == q0start]))
    lixmm1 = float((spd.lixm[spd.date == q1start]))
    lixmm0 = float((spd.lixm[spd.date == m0start]))
    q0lixm.append(float(lixm1))
    q0lixmv.append(float(lixm1 - lixm0))
    m0lixm.append(float(lixmm1))
    m0lixmv.append(float(lixmm1 - lixmm0))
    
    q1indstart = spw.nrow[spw.date == q1start]
    
    m0cstart = float(spw.c[q1indstart + 4])
    m0cend = q0cend  
    m0cv = ((m0cend/m0cstart)-1)*100
    m0c.append(float(m0cv))
    
    m0clist = spd.c[(spd.date >= m0start) & (spd.date<= q1start)]
    m0clist = m0clist.reset_index(drop=True)
    m0cvars = []
    for i in range(len(m0clist)):
        if i < len(m0clist)-1:
            m0cvars.append(float(((m0clist[i]/m0clist[i+1])-1)*100))
    m0cstd.append(float(np.std(m0cvars)))
    
    m0hlist = spd.h[(spd.date >= m0start) & (spd.date<= q1start)]
    m0hlist = m0hlist.reset_index(drop=True)
    m0hvars = []
    for i in range(len(m0hlist)):
        m0hvars.append(float(((m0hlist[i]/m0clist[i])-1)*100))
    m0high.append(float(np.mean(m0hvars)))
    
    m0llist = spd.l[(spd.date >= m0start) & (spd.date<= q1start)]
    m0llist = m0llist.reset_index(drop=True)
    m0lvars = []
    for i in range(len(m0llist)):
        m0lvars.append(float(((m0llist[i]/m0clist[i])-1)*100))
    m0low.append(float(np.mean(m0lvars)))
    
    m0svars = []
    for i in range(len(m0llist)):
        m0svars.append(float(((m0hlist[i]/m0llist[i])-1)*100))
    m0spanmean.append(float(np.mean(m0svars)))
    m0spanstd.append(float(np.std(m0svars)))

In [152]:
data = pd.DataFrame(index = exp, columns = ['q1c','q1cstd','q1high','q1low','q1range','q1middle','q1pos','q0c',
                                            'q0cstd','q0high','q0low','q0spanmean','q0spanstd','q0lixt','q0lixc72',
                                            'q0lixc288','q0lixd','q0lixm','q0lixtv','q0lixc72v','q0lixc288v',
                                            'q0lixdv','q0lixmv','m0c','m0cstd','m0high','m0low','m0spanmean',
                                            'm0spanstd','m0lixt','m0lixc72','m0lixc288','m0lixd','m0lixm','m0lixtv',
                                            'm0lixc72v','m0lixc288v','m0lixdv','m0lixmv'])

In [153]:
data.q1c = q1c
data.q1cstd = q1cstd
data.q1high = q1high
data.q1low = q1low
data.q1range = q1range
data.q1middle = q1middle
data.q1pos = q1pos
data.q0c = q0c
data.q0cstd = q0cstd
data.q0high = q0high
data.q0low = q0low
data.q0spanmean = q0spanmean
data.q0spanstd = q0spanstd
data.q0lixt = q0lixt
data.q0lixc72 = q0lixc72
data.q0lixc288 = q0lixc288
data.q0lixd = q0lixd
data.q0lixm = q0lixm
data.q0lixtv = q0lixtv
data.q0lixc72v = q0lixc72v
data.q0lixc288v = q0lixc288v
data.q0lixdv = q0lixdv
data.q0lixmv = q0lixmv
data.m0c = m0c
data.m0cstd = m0cstd
data.m0high = m0high
data.m0low = m0low
data.m0spanmean = m0spanmean
data.m0spanstd = m0spanstd
data.m0lixt = m0lixt
data.m0lixc72 = m0lixc72
data.m0lixc288 = m0lixc288
data.m0lixd = m0lixd
data.m0lixm = m0lixm
data.m0lixtv = m0lixtv
data.m0lixc72v = m0lixc72v
data.m0lixc288v = m0lixc288v
data.m0lixdv = m0lixdv
data.m0lixmv = m0lixmv

In [154]:
len(data)

141

In [155]:
data.head(3)

,q1c,q1cstd,q1high,q1low,q1range,q1middle,q1pos,q0c,q0cstd,q0high,...,m0lixt,m0lixc72,m0lixc288,m0lixd,m0lixm,m0lixtv,m0lixc72v,m0lixc288v,m0lixdv,m0lixmv
2015-03-20,1.808620,0.851452,2.257262,-3.765961,6.023222,-0.754349,85.102941,2.996910,0.920216,0.512111,...,7.285306,0,0,-5.667367,1,-0.514500,0,0,0.659982,0
2014-12-19,2.996910,0.920216,3.231700,-7.357244,10.588944,-2.062772,95.565378,2.421456,0.529902,0.296047,...,6.239703,0,0,-5.549377,1,-0.927099,0,0,0.082600,0
2014-09-19,2.421456,0.529902,2.470362,-2.715414,5.185776,-0.122526,98.113841,5.162011,0.599897,0.325310,...,8.935134,0,0,-7.054315,1,2.126565,0,0,-1.652266,0


In [156]:
data.tail(3)

,q1c,q1cstd,q1high,q1low,q1range,q1middle,q1pos,q0c,q0cstd,q0high,...,m0lixt,m0lixc72,m0lixc288,m0lixd,m0lixm,m0lixtv,m0lixc72v,m0lixc288v,m0lixdv,m0lixmv
1980-09-19,13.317554,0.848508,13.317554,0.000000,13.317554,6.658777,100.000000,11.484704,1.030380,1.051530,...,5.873997,0,-4.908781,-5.728637,1,2.612930,3.663194,-1.245588,-1.929926,0
1980-06-20,11.484704,1.030380,13.635035,-3.997651,17.632686,4.818692,75.609713,-4.907518,1.035450,1.184623,...,-3.365495,0,2.532203,4.825968,-1,-11.566643,-0.099965,2.432238,11.860217,-2
1980-03-21,-4.907518,1.035450,10.084586,-4.953987,15.038573,2.565300,-99.382002,-2.607047,0.860565,0.921324,...,3.839167,0,0.000000,-3.499354,1,2.506594,0.000000,-0.689172,-1.976642,1


In [157]:
data.to_csv('data.csv')
spd.to_csv('spdfinal.csv')
spw.to_csv('spwfinal.csv')

## Building alternative samples Dataset (predict last 2 monts of the quarter)

In [158]:
q1c = []
q1cstd = []
q1high = []
q1low = []
q1range = []
q1middle = []
q1pos = []
q0c = []
q0cstd = []
q0high = []
q0low = []
q0spanmean = []
q0spanstd = []
q0lixt = []
q0lixc72 = []
q0lixc288 = []
q0lixd = []
q0lixm = []
q0lixtv = []
q0lixc72v = []
q0lixc288v = []
q0lixdv = []
q0lixmv = []
m0c = []
m0cstd = []
m0high = []
m0low = []
m0spanmean = []
m0spanstd = []
m0lixt = []
m0lixc72 = []
m0lixc288 = []
m0lixd = []
m0lixm = []
m0lixtv = []
m0lixc72v = []
m0lixc288v = []
m0lixdv = []
m0lixmv = []

In [159]:
for q1end in exp:
    q1start = q1end - DateOffset(weeks = 8)
    m0start = q1start - DateOffset(weeks = 5)
    q0start = q1start - DateOffset(weeks = 13)
    q1indend = spw.nrow[spw.date == q1end]
    
    q1cstart = float(spw.c[q1indend + 8])
    q1cend = float(spw.c[q1indend])    
    q1cv = ((q1cend/q1cstart)-1)*100
    q1c.append(q1cv)   
    
    q1clist = spd.c[(spd.date >= q1start) & (spd.date<= q1end)]
    q1clist = q1clist.reset_index(drop=True)
    q1cvars = []
    for i in range(len(q1clist)):
        if i < len(q1clist)-1:
            q1cvars.append(((q1clist[i]/q1clist[i+1])-1)*100)
    q1cstd.append(np.std(q1cvars))
    
    q1chigh = ((np.max(q1clist)/q1cstart)-1)*100
    q1high.append(q1chigh)
    
    q1clow = ((np.min(q1clist)/q1cstart)-1)*100
    q1low.append(q1clow)
    
    q1crange = q1chigh - q1clow
    q1range.append(q1crange)
    
    q1cmiddle = q1chigh - (q1crange / 2)
    q1middle.append(q1cmiddle)
    
    q1cpos = (((q1cv - q1clow)/q1crange)-0.5)*200
    q1pos.append(q1cpos)
      
    q1indstart = spw.nrow[spw.date == q1start]
    
    q0cstart = float(spw.c[q1indstart + 13])
    q0cend = float(spw.c[q1indstart])    
    q0cv = ((q0cend/q0cstart)-1)*100
    q0c.append(q0cv)
    
    q0clist = spd.c[(spd.date >= q0start) & (spd.date<= q1start)]
    q0clist = q0clist.reset_index(drop=True)
    q0cvars = []
    for i in range(len(q0clist)):
        if i < len(q0clist)-1:
            q0cvars.append(((q0clist[i]/q0clist[i+1])-1)*100)
    q0cstd.append(np.std(q0cvars))
    
    q0hlist = spd.h[(spd.date >= q0start) & (spd.date<= q1start)]
    q0hlist = q0hlist.reset_index(drop=True)
    q0hvars = []
    for i in range(len(q0hlist)):
        q0hvars.append(((q0hlist[i]/q0clist[i])-1)*100)
    q0high.append(np.mean(q0hvars))
    
    q0llist = spd.l[(spd.date >= q0start) & (spd.date<= q1start)]
    q0llist = q0llist.reset_index(drop=True)
    q0lvars = []
    for i in range(len(q0llist)):
        q0lvars.append(((q0llist[i]/q0clist[i])-1)*100)
    q0low.append(np.mean(q0lvars))
    
    q0svars = []
    for i in range(len(q0llist)):
        q0svars.append(((q0hlist[i]/q0llist[i])-1)*100)
    q0spanmean.append(np.mean(q0svars))
    q0spanstd.append(np.std(q0svars))

    lixt1 = float((spw.lixt[spw.date == q1start]))
    lixt0 = float((spw.lixt[spw.date == q0start]))
    lixtm1 = float((spd.lixt[spd.date == q1start]))
    lixtm0 = float((spd.lixt[spd.date == m0start]))
    q0lixt.append(float(lixt1))
    q0lixtv.append(float(lixt1 - lixt0))
    m0lixt.append(float(lixtm1))
    m0lixtv.append(float(lixtm1 - lixtm0))
    
    lixc721 = float((spw.lixc72[spw.date == q1start]))
    lixc720 = float((spw.lixc72[spw.date == q0start]))
    lixc72m1 = float((spd.lixc72[spd.date == q1start]))
    lixc72m0 = float((spd.lixc72[spd.date == m0start]))
    q0lixc72.append(float(lixc721))
    q0lixc72v.append(float(lixc721 - lixc720))
    m0lixc72.append(float(lixc72m1))
    m0lixc72v.append(float(lixc72m1 - lixc72m0))
    
    lixc2881 = float((spw.lixc288[spw.date == q1start]))
    lixc2880 = float((spw.lixc288[spw.date == q0start]))
    lixc288m1 = float((spd.lixc288[spd.date == q1start]))
    lixc288m0 = float((spd.lixc288[spd.date == m0start]))
    q0lixc288.append(float(lixc2881))
    q0lixc288v.append(float(lixc2881 - lixc2880))
    m0lixc288.append(float(lixc288m1))
    m0lixc288v.append(float(lixc288m1 - lixc288m0))
    
    lixd1 = float((spw.lixd[spw.date == q1start]))
    lixd0 = float((spw.lixd[spw.date == q0start]))
    lixdm1 = float((spd.lixd[spd.date == q1start]))
    lixdm0 = float((spd.lixd[spd.date == m0start]))
    q0lixd.append(float(lixd1))
    q0lixdv.append(float(lixd1 - lixd0))
    m0lixd.append(float(lixdm1))
    m0lixdv.append(float(lixdm1 - lixdm0))
    
    lixm1 = float((spw.lixm[spw.date == q1start]))
    lixm0 = float((spw.lixm[spw.date == q0start]))
    lixmm1 = float((spd.lixm[spd.date == q1start]))
    lixmm0 = float((spd.lixm[spd.date == m0start]))
    q0lixm.append(float(lixm1))
    q0lixmv.append(float(lixm1 - lixm0))
    m0lixm.append(float(lixmm1))
    m0lixmv.append(float(lixmm1 - lixmm0))
    
    q1indstart = spw.nrow[spw.date == q1start]
    
    m0cstart = float(spw.c[q1indstart + 4])
    m0cend = q0cend  
    m0cv = ((m0cend/m0cstart)-1)*100
    m0c.append(float(m0cv))
    
    m0clist = spd.c[(spd.date >= m0start) & (spd.date<= q1start)]
    m0clist = m0clist.reset_index(drop=True)
    m0cvars = []
    for i in range(len(m0clist)):
        if i < len(m0clist)-1:
            m0cvars.append(float(((m0clist[i]/m0clist[i+1])-1)*100))
    m0cstd.append(float(np.std(m0cvars)))
    
    m0hlist = spd.h[(spd.date >= m0start) & (spd.date<= q1start)]
    m0hlist = m0hlist.reset_index(drop=True)
    m0hvars = []
    for i in range(len(m0hlist)):
        m0hvars.append(float(((m0hlist[i]/m0clist[i])-1)*100))
    m0high.append(float(np.mean(m0hvars)))
    
    m0llist = spd.l[(spd.date >= m0start) & (spd.date<= q1start)]
    m0llist = m0llist.reset_index(drop=True)
    m0lvars = []
    for i in range(len(m0llist)):
        m0lvars.append(float(((m0llist[i]/m0clist[i])-1)*100))
    m0low.append(float(np.mean(m0lvars)))
    
    m0svars = []
    for i in range(len(m0llist)):
        m0svars.append(float(((m0hlist[i]/m0llist[i])-1)*100))
    m0spanmean.append(float(np.mean(m0svars)))
    m0spanstd.append(float(np.std(m0svars)))

In [160]:
data2 = pd.DataFrame(index = exp, columns = ['q1c','q1cstd','q1high','q1low','q1range','q1middle','q1pos','q0c',
                                            'q0cstd','q0high','q0low','q0spanmean','q0spanstd','q0lixt','q0lixc72',
                                            'q0lixc288','q0lixd','q0lixm','q0lixtv','q0lixc72v','q0lixc288v',
                                            'q0lixdv','q0lixmv','m0c','m0cstd','m0high','m0low','m0spanmean',
                                            'm0spanstd','m0lixt','m0lixc72','m0lixc288','m0lixd','m0lixm','m0lixtv',
                                            'm0lixc72v','m0lixc288v','m0lixdv','m0lixmv'])

In [161]:
data2.q1c = q1c
data2.q1cstd = q1cstd
data2.q1high = q1high
data2.q1low = q1low
data2.q1range = q1range
data2.q1middle = q1middle
data2.q1pos = q1pos
data2.q0c = q0c
data2.q0cstd = q0cstd
data2.q0high = q0high
data2.q0low = q0low
data2.q0spanmean = q0spanmean
data2.q0spanstd = q0spanstd
data2.q0lixt = q0lixt
data2.q0lixc72 = q0lixc72
data2.q0lixc288 = q0lixc288
data2.q0lixd = q0lixd
data2.q0lixm = q0lixm
data2.q0lixtv = q0lixtv
data2.q0lixc72v = q0lixc72v
data2.q0lixc288v = q0lixc288v
data2.q0lixdv = q0lixdv
data2.q0lixmv = q0lixmv
data2.m0c = m0c
data2.m0cstd = m0cstd
data2.m0high = m0high
data2.m0low = m0low
data2.m0spanmean = m0spanmean
data2.m0spanstd = m0spanstd
data2.m0lixt = m0lixt
data2.m0lixc72 = m0lixc72
data2.m0lixc288 = m0lixc288
data2.m0lixd = m0lixd
data2.m0lixm = m0lixm
data2.m0lixtv = m0lixtv
data2.m0lixc72v = m0lixc72v
data2.m0lixc288v = m0lixc288v
data2.m0lixdv = m0lixdv
data2.m0lixmv = m0lixmv

In [162]:
len(data2)

141

In [163]:
data2.head(3)

,q1c,q1cstd,q1high,q1low,q1range,q1middle,q1pos,q0c,q0cstd,q0high,...,m0lixt,m0lixc72,m0lixc288,m0lixd,m0lixm,m0lixtv,m0lixc72v,m0lixc288v,m0lixdv,m0lixmv
2015-03-20,2.742932,0.824285,3.195691,-2.769740,5.965431,0.212975,84.820584,4.440650,0.812940,0.445199,...,5.122899,0.000000,0.000000,-3.827465,1,-2.162407,0.000000,0.000000,1.839902,0
2014-12-19,5.399116,0.756634,5.639382,-0.150157,5.789538,2.744612,91.699999,-0.695533,0.820467,0.487591,...,4.232587,-1.387678,-1.387678,-2.862472,1,-2.007115,-1.387678,-1.387678,2.686906,0
2014-09-19,1.620554,0.548310,1.669077,-3.476148,5.145225,-0.903535,98.113841,6.168291,0.462849,0.229158,...,7.621349,0.000000,0.000000,-6.110578,1,-1.313785,0.000000,0.000000,0.943736,0


In [164]:
data2.tail(3)

,q1c,q1cstd,q1high,q1low,q1range,q1middle,q1pos,q0c,q0cstd,q0high,...,m0lixt,m0lixc72,m0lixc288,m0lixd,m0lixm,m0lixtv,m0lixc72v,m0lixc288v,m0lixdv,m0lixmv
1980-09-19,7.012751,0.867583,7.012751,-0.033119,7.045870,3.489816,100.000000,14.853551,0.782343,0.884217,...,12.647615,0,0.000000,-9.007649,1,6.773618,0,4.908781,-3.279013,0
1980-06-20,8.463288,0.761325,10.555342,-0.418413,10.973755,5.068465,61.871684,-7.437723,1.227000,1.308383,...,-0.211931,0,0.968519,1.125175,-1,3.153564,0,-1.563684,-3.700793,0
1980-03-21,-9.946310,1.114831,4.251387,-9.990317,14.241704,-2.869465,-99.382002,12.966094,0.781962,0.853927,...,8.780949,0,0.000000,-7.590476,1,4.941782,0,0.000000,-4.091121,0


In [165]:
ydata2 = pd.DataFrame(index = exp, columns = ['q1c','q1cstd','q1high','q1low','q1range','q1middle','q1pos'])

In [166]:
ydata2.q1c = q1c
ydata2.q1cstd = q1cstd
ydata2.q1high = q1high
ydata2.q1low = q1low
ydata2.q1range = q1range
ydata2.q1middle = q1middle
ydata2.q1pos = q1pos
ydata2.head(3)

,q1c,q1cstd,q1high,q1low,q1range,q1middle,q1pos
2015-03-20,2.742932,0.824285,3.195691,-2.769740,5.965431,0.212975,84.820584
2014-12-19,5.399116,0.756634,5.639382,-0.150157,5.789538,2.744612,91.699999
2014-09-19,1.620554,0.548310,1.669077,-3.476148,5.145225,-0.903535,98.113841


In [167]:
data2.to_csv('data2.csv')

## Data normalization

In [168]:
datan = data.copy()
datan2 = data2.copy()

In [169]:
for cols in datan.keys():
    maxcols = max(datan[cols])
    mincols = min(datan[cols])
    spancols = maxcols - mincols
    meancols = maxcols - (spancols / 2)
    for n in range(len(datan)):
        if mincols >= 0 or maxcols <= 0:
            datan[cols][n] = (datan[cols][n] - meancols) / (spancols / 2)
        else:
            if datan[cols][n] < 0:
                datan[cols][n] = datan[cols][n] / abs(mincols)
            if datan[cols][n] > 0:
                datan[cols][n] = datan[cols][n] / maxcols

In [170]:
for cols in datan.keys():
    maxcols = max(datan[cols])
    mincols = min(datan[cols])
    print cols, mincols, maxcols

q1c -1.0 1.0
q1cstd -1.0 1.0
q1high -1.0 1.0
q1low -1.0 1.0
q1range -1.0 1.0
q1middle -1.0 1.0
q1pos -1.0 1.0
q0c -1.0 1.0
q0cstd -1.0 1.0
q0high -1.0 1.0
q0low -1.0 1.0
q0spanmean -1.0 1.0
q0spanstd -1.0 1.0
q0lixt -1.0 1.0
q0lixc72 -1.0 1.0
q0lixc288 -1.0 1.0
q0lixd -1.0 1.0
q0lixm -1.0 1.0
q0lixtv -1.0 1.0
q0lixc72v -1.0 1.0
q0lixc288v -1.0 1.0
q0lixdv -1.0 1.0
q0lixmv -1.0 1.0
m0c -1.0 1.0
m0cstd -1.0 1.0
m0high -1.0 1.0
m0low -1.0 1.0
m0spanmean -1.0 1.0
m0spanstd -1.0 1.0
m0lixt -1.0 1.0
m0lixc72 -1.0 1.0
m0lixc288 -1.0 1.0
m0lixd -1.0 1.0
m0lixm -1.0 1.0
m0lixtv -1.0 1.0
m0lixc72v -1.0 1.0
m0lixc288v -1.0 1.0
m0lixdv -1.0 1.0
m0lixmv -1.0 1.0


In [171]:
datan.head(3)

,q1c,q1cstd,q1high,q1low,q1range,q1middle,q1pos,q0c,q0cstd,q0high,...,m0lixt,m0lixc72,m0lixc288,m0lixd,m0lixm,m0lixtv,m0lixc72v,m0lixc288v,m0lixdv,m0lixmv
2015-03-20,0.091034,-0.784030,-0.804717,0.811930,-0.867391,-0.037672,0.851029,0.150845,-0.749593,-0.793867,...,0.402011,0,0,-0.406701,1,-0.031447,0,0,0.030712,0
2014-12-19,0.150845,-0.749593,-0.720415,0.632583,-0.616813,-0.103014,0.955654,0.121880,-0.945065,-0.959844,...,0.344314,0,0,-0.398234,1,-0.056665,0,0,0.003844,0
2014-09-19,0.121880,-0.945065,-0.786281,0.864394,-0.913353,-0.006119,0.981138,0.259821,-0.910011,-0.937364,...,0.493051,0,0,-0.506231,1,0.168571,0,0,-0.078084,0


In [172]:
data.head(3)

,q1c,q1cstd,q1high,q1low,q1range,q1middle,q1pos,q0c,q0cstd,q0high,...,m0lixt,m0lixc72,m0lixc288,m0lixd,m0lixm,m0lixtv,m0lixc72v,m0lixc288v,m0lixdv,m0lixmv
2015-03-20,1.808620,0.851452,2.257262,-3.765961,6.023222,-0.754349,85.102941,2.996910,0.920216,0.512111,...,7.285306,0,0,-5.667367,1,-0.514500,0,0,0.659982,0
2014-12-19,2.996910,0.920216,3.231700,-7.357244,10.588944,-2.062772,95.565378,2.421456,0.529902,0.296047,...,6.239703,0,0,-5.549377,1,-0.927099,0,0,0.082600,0
2014-09-19,2.421456,0.529902,2.470362,-2.715414,5.185776,-0.122526,98.113841,5.162011,0.599897,0.325310,...,8.935134,0,0,-7.054315,1,2.126565,0,0,-1.652266,0


In [173]:
for cols in datan2.keys():
    maxcols = max(datan2[cols])
    mincols = min(datan2[cols])
    spancols = maxcols - mincols
    meancols = maxcols - (spancols / 2)
    for n in range(len(datan2)):
        if mincols >= 0 or maxcols <= 0:
            datan2[cols][n] = (datan2[cols][n] - meancols) / (spancols / 2)
        else:
            if datan2[cols][n] < 0:
                datan2[cols][n] = datan2[cols][n] / abs(mincols)
            if datan2[cols][n] > 0:
                datan2[cols][n] = datan2[cols][n] / maxcols

In [174]:
for cols in datan2.keys():
    maxcols = max(datan2[cols])
    mincols = min(datan2[cols])
    print cols, mincols, maxcols

q1c -1.0 1.0
q1cstd -1.0 1.0
q1high -1.0 1.0
q1low -1.0 1.0
q1range -1.0 1.0
q1middle -1.0 1.0
q1pos -1.0 1.0
q0c -1.0 1.0
q0cstd -1.0 1.0
q0high -1.0 1.0
q0low -1.0 1.0
q0spanmean -1.0 1.0
q0spanstd -1.0 1.0
q0lixt -1.0 1.0
q0lixc72 -1.0 1.0
q0lixc288 -1.0 1.0
q0lixd -1.0 1.0
q0lixm -1.0 1.0
q0lixtv -1.0 1.0
q0lixc72v -1.0 1.0
q0lixc288v -1.0 1.0
q0lixdv -1.0 1.0
q0lixmv -1.0 1.0
m0c -1.0 1.0
m0cstd -1.0 1.0
m0high -1.0 1.0
m0low -1.0 1.0
m0spanmean -1.0 1.0
m0spanstd -1.0 1.0
m0lixt -1.0 1.0
m0lixc72 -1.0 1.0
m0lixc288 -1.0 1.0
m0lixd -1.0 1.0
m0lixm -1.0 1.0
m0lixtv -1.0 1.0
m0lixc72v -1.0 1.0
m0lixc288v -1.0 1.0
m0lixdv -1.0 1.0
m0lixmv -1.0 1.0


In [175]:
datan2.head(3)

,q1c,q1cstd,q1high,q1low,q1range,q1middle,q1pos,q0c,q0cstd,q0high,...,m0lixt,m0lixc72,m0lixc288,m0lixd,m0lixm,m0lixtv,m0lixc72v,m0lixc288v,m0lixdv,m0lixmv
2015-03-20,0.157456,-0.760874,-0.633108,0.721706,-0.737782,0.024451,0.848206,0.178477,-0.732095,-0.756883,...,0.264939,0.000000,0.00000,-0.251692,1,-0.084272,0.000000,0.000000,0.048247,0
2014-12-19,0.309932,-0.796206,-0.352552,0.984913,-0.751115,0.315104,0.917000,-0.022962,-0.727418,-0.709178,...,0.218895,-0.463666,-0.10246,-0.188234,1,-0.078220,-0.463666,-0.105079,0.070457,0
2014-09-19,0.093027,-0.905005,-0.808376,0.650728,-0.799954,-0.096806,0.981138,0.247914,-0.949596,-1.000000,...,0.394150,0.000000,0.00000,-0.401828,1,-0.051200,0.000000,0.000000,0.024747,0


In [176]:
data2.head(3)

,q1c,q1cstd,q1high,q1low,q1range,q1middle,q1pos,q0c,q0cstd,q0high,...,m0lixt,m0lixc72,m0lixc288,m0lixd,m0lixm,m0lixtv,m0lixc72v,m0lixc288v,m0lixdv,m0lixmv
2015-03-20,2.742932,0.824285,3.195691,-2.769740,5.965431,0.212975,84.820584,4.440650,0.812940,0.445199,...,5.122899,0.000000,0.000000,-3.827465,1,-2.162407,0.000000,0.000000,1.839902,0
2014-12-19,5.399116,0.756634,5.639382,-0.150157,5.789538,2.744612,91.699999,-0.695533,0.820467,0.487591,...,4.232587,-1.387678,-1.387678,-2.862472,1,-2.007115,-1.387678,-1.387678,2.686906,0
2014-09-19,1.620554,0.548310,1.669077,-3.476148,5.145225,-0.903535,98.113841,6.168291,0.462849,0.229158,...,7.621349,0.000000,0.000000,-6.110578,1,-1.313785,0.000000,0.000000,0.943736,0


In [177]:
datan.to_csv('datan.csv')
datan2.to_csv('datan2.csv')

## Overbought/Oversold Indicator using Options Open Interest

In [182]:
options = read_csv('/Users/marco/Documents/Doc/options.csv', delimiter=',', header=0)
options.rename(columns={'Unnamed: 0':'nrow'}, inplace=True)
len(options)

246258

In [183]:
options.head(3)

,nrow,date,exp,strike,pc,oi,vol,h,l,o,c,sp
0,0,1990-01-05 00:00:00,1990-12-22 00:00:00,250,C,320,0,0,0,0,0,352.2
1,1,1990-01-05 00:00:00,1990-12-22 00:00:00,250,P,4045,0,0,0,0,0,352.2
2,2,1990-01-05 00:00:00,1990-12-22 00:00:00,275,C,3,0,0,0,0,0,352.2


In [184]:
options.tail(3)

,nrow,date,exp,strike,pc,oi,vol,h,l,o,c,sp
246255,246255,2015-05-29 00:00:00,2016-03-18 00:00:00,2800,P,2,0,0,0,0,0,2107.39
246256,246256,2015-05-29 00:00:00,2016-03-18 00:00:00,2900,P,12,0,0,0,0,0,2107.39
246257,246257,2015-05-29 00:00:00,2016-03-18 00:00:00,3000,P,12,0,0,0,0,0,2107.39


In [185]:
options['date'] = pd.to_datetime(options['date'])
options['exp'] = pd.to_datetime(options['exp'])

##Setting expiration and date

In [186]:
expd = dp.parse("2009/03/21")
d = dp.parse("2009/03/09")

In [187]:
optexp = options[options.exp == expd]
len(optexp)

740

In [188]:
optd = optexp[optexp.date == d]
optd = optd.reset_index(drop=True)
len(optd)

0

In [189]:
price = optd.sp[0]
price

IndexError: index out of bounds

In [ ]:
#price = 2040
strikes = []
pcr = []
diff = []

In [ ]:
if price%25 == 0:
    start = price - 25
else:
    start = (int(price / 25)) * 25
strikes = [(start - 25), start, (start + 25), (start + 50)]
strikes    

In [ ]:
for s in strikes:
    opts = optd[optd.strike == s]
    oic = float(np.max(opts.oi[opts.pc == 'C']))
    oip = float(np.max(opts.oi[opts.pc == 'P']))
    ratio = ((oip/(oic + oip))*2)-1
    pcr.append(ratio)
    if s < price:
        diff.append((((s - 25)/price)-1)*100)
    elif s > price:
        diff.append((((s + 25)/price)-1)*100)
pcr

In [ ]:
diff

In [ ]:
obs = 0
mult = 3
if pcr[0] < 0:
    obs += (abs(pcr[0] * mult) * diff[0])
if pcr[1] < 0:
    obs += (abs(pcr[1] * mult) * diff[1])
if pcr[2] > 0:
    obs += (abs(pcr[2] * mult) * diff[2])
if pcr[3] > 0:
    obs += (abs(pcr[3] * mult) * diff[3])

In [ ]:
(abs(pcr[0]*4) * diff[0])

In [ ]:
(abs(pcr[1]*4) * diff[1])

In [ ]:
(abs(pcr[2]*4) * diff[2])

In [ ]:
(abs(pcr[3]*4) * diff[3])

In [ ]:
obs

## Predicting S&P 500 quarterly fluctuation spans (to be completed)

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
%pylab inline

In [ ]:
hie_data = datan
corr_matrix = np.corrcoef(hie_data)
sm.graphics.plot_corr(corr_matrix, xnames=list(datan.columns.values))

In [ ]:
corr_matrix

In [ ]:
num_clusters=3
km = KMeans(init='k-means++', max_iter=50, n_init=1, verbose=1, n_jobs=-1, n_clusters=num_clusters)
km.fit(ydata2)

In [ ]:
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = list(ydata2.columns.values)
for i in range(num_clusters):
    print "Cluster %d:" % i,
    for ind in order_centroids[i, :50]:
        print ' %s' % terms[ind] + ",",
    print "\n"

## Creating datasets for predictions (starting from data2 and after features selection)

In [ ]:
complete = data2.copy()
completen = datan2.copy()

In [ ]:
yall = complete.drop(['q1c','q1cstd','q0c','q0cstd','q0high','q0low','q0spanmean','q0spanstd','q0lixt','q0lixc72',
                      'q0lixc288','q0lixd','q0lixm','q0lixtv','q0lixc72v','q0lixc288v','q0lixdv','q0lixmv','m0c',
                      'm0cstd','m0high','m0low','m0spanmean','m0spanstd','m0lixt','m0lixc72','m0lixc288','m0lixd',
                      'm0lixm','m0lixtv','m0lixc72v','m0lixc288v','m0lixdv','m0lixmv'], 1)
yalln = completen.drop(['q1c','q1cstd','q0c','q0cstd','q0high','q0low','q0spanmean','q0spanstd','q0lixt','q0lixc72',
                      'q0lixc288','q0lixd','q0lixm','q0lixtv','q0lixc72v','q0lixc288v','q0lixdv','q0lixmv','m0c',
                      'm0cstd','m0high','m0low','m0spanmean','m0spanstd','m0lixt','m0lixc72','m0lixc288','m0lixd',
                      'm0lixm','m0lixtv','m0lixc72v','m0lixc288v','m0lixdv','m0lixmv'], 1)
yall.head(3)

In [ ]:
xall = complete.drop(['q1c','q1cstd','q1high','q1low','q1range','q1middle','q1pos','q0lixc72v','q0lixc288v','m0lixc288',
                    'm0lixtv','m0lixc72v','m0lixc288v','m0lixdv','m0lixmv'], 1)
xalln = completen.drop(['q1c','q1cstd','q1high','q1low','q1range','q1middle','q1pos','q0lixc72v','q0lixc288v','m0lixc288',
                    'm0lixtv','m0lixc72v','m0lixc288v','m0lixdv','m0lixmv'], 1)
xall.head(3)

###Q1Range

In [ ]:
yr = yall.q1range
xr = xall.drop(['q0lixc288','q0lixc72','m0lixc72'], 1)
yrn = yalln.q1range
xrn = xalln.drop(['q0lixc288','q0lixc72','m0lixc72'], 1)

In [ ]:
xr.columns.values

###Q1Low

In [ ]:
yl = yall.q1low
xl = xall.drop(['q0lixc288','q0lixt','m0c','m0spanstd','m0lixm'], 1)
yln = yalln.q1low
xln = xalln.drop(['q0lixc288','q0lixt','m0c','m0spanstd','m0lixm'], 1)

In [ ]:
xl.columns.values

###Q1High

In [ ]:
yh = yall.q1high
xh = xall.drop(['q0lixc288','q0lixt','m0c','m0spanstd','m0lixm'], 1)
yhn = yalln.q1high
xhn = xalln.drop(['q0lixc288','q0lixt','m0c','m0spanstd','m0lixm'], 1)

In [ ]:
xh.columns.values

###Q1Middle

In [ ]:
ym = yall.q1middle
xm = xall.drop(['q0c','q0cstd','q0low','q0spanmean','q0spanstd','q0lixt','q0lixc72',
               'q0lixd','q0lixm','q0lixtv','q0lixdv','q0lixmv','m0c','m0cstd','m0low','m0high','m0spanmean',
               'm0spanstd','m0lixd','m0lixm'], 1)
ymn = yalln.q1middle
xmn = xalln.drop(['q0c','q0cstd','q0low','q0spanmean','q0spanstd','q0lixt','q0lixc72',
               'q0lixd','q0lixm','q0lixtv','q0lixdv','q0lixmv','m0c','m0cstd','m0low','m0high','m0spanmean',
               'm0spanstd','m0lixd','m0lixm'], 1)

In [ ]:
xm.columns.values

###Q1Pos

In [ ]:
yp = yall.q1pos
xp = xall.drop(['q0c','q0cstd','q0high','q0low','q0spanstd','q0lixt','q0lixc72','q0lixd',
               'q0lixm','q0lixtv','q0lixdv','q0lixmv','m0c','m0cstd','m0low','m0spanmean','m0spanstd','m0lixm'], 1)
ypn = yalln.q1pos
xpn = xalln.drop(['q0c','q0cstd','q0high','q0low','q0spanstd','q0lixt','q0lixc72','q0lixd',
               'q0lixm','q0lixtv','q0lixdv','q0lixmv','m0c','m0cstd','m0low','m0spanmean','m0spanstd','m0lixm'], 1)

In [ ]:
xp.columns.values

##Supervised learning predictions

In [ ]:
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from scipy.stats import linregress
from pylab import *
from urllib import urlopen
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from __future__ import division
from sklearn import cross_validation
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn import tree
from sklearn.externals.six import StringIO
import os
from sklearn.externals.six import StringIO  
import pydot
from sklearn.cross_validation import StratifiedKFold
from sklearn import linear_model
from sklearn.cross_validation import cross_val_score

In [ ]:
testsize = 0.2
#clf = RandomForestClassifier()
clf = DecisionTreeClassifier()
#clf = GaussianNB()
#clf = SVC()
#clf = LogisticRegression()

###Q1low

In [ ]:
X = xl
y = yl
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=testsize, random_state=0)
Xn = xln
yn = yln
Xn_train, Xn_test, yn_train, yn_test = cross_validation.train_test_split(Xn, yn, test_size=testsize, random_state=0)

In [ ]:
print X.shape, X_train.shape, X_test.shape

In [ ]:
model = clf.fit(X_train, y_train)
modeln = clf.fit(Xn_train, yn_train)

In [ ]:
q1lpred = model.predict(X_test)
q1lpredn = model.predict(Xn_test)

In [ ]:
diff = (q1lpred - y_test)
diffn = (q1lpredn - yn_test)

In [ ]:
print np.mean(abs(diff)), np.max(abs(diff))
print np.mean(abs(diffn)), np.max(abs(diffn))

In [ ]:
pred = pd.DataFrame(columns = ['actual','predicted','difference'])
pred.actual = y_test
pred.predicted = q1lpred
pred.difference = diff
pred.head(3)

###Q1high

In [ ]:
X = xh
y = yh
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=testsize, random_state=0)
Xn = xhn
yn = yhn
Xn_train, Xn_test, yn_train, yn_test = cross_validation.train_test_split(Xn, yn, test_size=testsize, random_state=0)

In [ ]:
print X.shape, X_train.shape, X_test.shape

In [ ]:
model = clf.fit(X_train, y_train)
modeln = clf.fit(Xn_train, yn_train)

In [ ]:
q1hpred = model.predict(X_test)
q1hpredn = model.predict(Xn_test)

In [ ]:
diff = (q1hpred - y_test)
diffn = (q1hpredn - yn_test)

In [ ]:
print np.mean(abs(diff)), np.max(abs(diff))
print np.mean(abs(diffn)), np.max(abs(diffn))

###Q1range

In [ ]:
X = xr
y = yr
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=testsize, random_state=0)
Xn = xrn
yn = yrn
Xn_train, Xn_test, yn_train, yn_test = cross_validation.train_test_split(Xn, yn, test_size=testsize, random_state=0)

In [ ]:
print X.shape, X_train.shape, X_test.shape

In [ ]:
model = clf.fit(X_train, y_train)
modeln = clf.fit(Xn_train, yn_train)

In [ ]:
q1rpred = model.predict(X_test)
q1rpredn = model.predict(Xn_test)

In [ ]:
diff = (q1rpred - y_test)
diffn = (q1rpredn - yn_test)

In [ ]:
print np.mean(abs(diff)), np.max(abs(diff))
print np.mean(abs(diffn)), np.max(abs(diffn))

###Q1middle

In [ ]:
X = xm
y = ym
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=testsize, random_state=0)
Xn = xmn
yn = ymn
Xn_train, Xn_test, yn_train, yn_test = cross_validation.train_test_split(Xn, yn, test_size=testsize, random_state=0)

In [ ]:
print X.shape, X_train.shape, X_test.shape

In [ ]:
model = clf.fit(X_train, y_train)
modeln = clf.fit(Xn_train, yn_train)

In [ ]:
q1mpred = model.predict(X_test)
q1mpredn = model.predict(Xn_test)

In [ ]:
diff = (q1mpred - y_test)
diffn = (q1mpredn - yn_test)

In [ ]:
print np.mean(abs(diff)), np.max(abs(diff))
print np.mean(abs(diffn)), np.max(abs(diffn))

###Q1pos

In [ ]:
X = xp
y = yp
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=testsize, random_state=0)
Xn = xpn
yn = ypn
Xn_train, Xn_test, yn_train, yn_test = cross_validation.train_test_split(Xn, yn, test_size=testsize, random_state=0)

In [ ]:
print X.shape, X_train.shape, X_test.shape

In [ ]:
model = clf.fit(X_train, y_train)
modeln = clf.fit(Xn_train, yn_train)

In [ ]:
q1ppred = model.predict(X_test)
q1ppredn = model.predict(Xn_test)

In [ ]:
diff = (q1ppred - y_test)
diffn = (q1ppredn - yn_test)

In [ ]:
print np.mean(abs(diff)), np.max(abs(diff))
print np.mean(abs(diffn)), np.max(abs(diffn))